In [1]:
import ru_core_news_lg
import spacy as sp
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

from function import *

nlp = sp.load('ru_core_news_lg')

Попробуем в качестве основного текста взять мифы Древней Греции.

In [42]:
text = "Была у великой богини Деметры юная прекрасная дочь Персефона. Отцом Персефоны был сам великий сын Крона, громовержец Зевс. Однажды прекрасная Персефона вместе со своими подругами, океанидами, беззаботно резвилась в цветущей Нисейской долине. Подобно легкокрылой бабочке перебегала юная дочь Деметры от цветка к цветку. Она рвала пышные розы, душистые фиалки, белоснежные лилии и красные гиацинты. Беспечно резвилась Персефона, не ведая той судьбы, которую назначил ей отец ее Зевс. Не думала Персефона, что не скоро увидит она опять ясный свет солнца, не скоро будет любоваться цветами и вдыхать их сладкий аромат. Зевс отдал ее в жены мрачному своему брату Аиду, властителю царства теней умерших, и с ним должна была жить Персефона во мраке подземного царства, лишенная света и горячего южного солнца."

doc = nlp(text)
sents = list(doc.sents)
for sent in sents:
    print(sent)

Была у великой богини Деметры юная прекрасная дочь Персефона.
Отцом Персефоны был сам великий сын Крона, громовержец Зевс.
Однажды прекрасная Персефона вместе со своими подругами, океанидами, беззаботно резвилась в цветущей Нисейской долине.
Подобно легкокрылой бабочке перебегала юная дочь Деметры от цветка к цветку.
Она рвала пышные розы, душистые фиалки, белоснежные лилии и красные гиацинты.
Беспечно резвилась Персефона, не ведая той судьбы, которую назначил ей отец ее Зевс.
Не думала Персефона, что не скоро увидит она опять ясный свет солнца, не скоро будет любоваться цветами и вдыхать их сладкий аромат.
Зевс отдал ее в жены мрачному своему брату Аиду, властителю царства теней умерших, и с ним должна была жить Персефона во мраке подземного царства, лишенная света и горячего южного солнца.


In [3]:
displacy.render(sents[0], style='dep')
nice_print(sents[0])

text           pos_       dep_        
_______________________________________________________
Была           AUX        cop         
у              ADP        case        
великой        ADJ        amod        
богини         NOUN       ROOT        
Деметры        PROPN      appos       
юная           ADJ        amod        
прекрасная     ADJ        amod        
дочь           NOUN       nsubj       
Персефона      PROPN      nmod        
.              PUNCT      punct       
_______________________________________________________


Для облегчения задания вопросов к тексту, исключим все прилагательные. 

In [4]:
displacy.render(doc, style = 'ent')

Как видно, смысл предложений от этого не искажается. 

In [5]:
def delete_other(sent):
    ans = ""
    for token in sent:
        if token.ent_iob == 3:
            ans = ans + token.text + " "
        elif token.pos_ != 'ADJ':
            ans = ans + token.text + " "
    return nlp(ans.strip().replace(" .", "."). replace(" ,", ",").replace("[ * ]", ""))

sents = [delete_other(i) for i in sents]

# sents - массив хранящий все предложения
print(sents[0])

Была у богини Деметры дочь Персефона.


In [6]:
print(list(sents))

[Была у богини Деметры дочь Персефона., Отцом Персефоны был сын Крона, громовержец Зевс., Однажды Персефона вместе со своими подругами, океанидами, беззаботно резвилась в цветущей Нисейской долине ., Подобно бабочке перебегала дочь Деметры от цветка к цветку., Она рвала розы, фиалки, лилии и гиацинты., Беспечно резвилась Персефона, не ведая той судьбы, которую назначил ей отец ее Зевс., Не думала Персефона, что не скоро увидит она опять свет солнца, не скоро будет любоваться цветами и вдыхать их аромат., Зевс отдал ее в жены своему брату Аиду, властителю царства теней умерших, и с ним была жить Персефона во мраке царства, лишенная света и солнца.]


In [7]:
displacy.render(sents, style = 'ent')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


<b>Замечание:</b> во втором предложении имя "Персефона" перестало иметь метку Persona.

Для формирования вопросов будем использовать обход деревьев зависимостей, выделение именнованных сущностей и паттерны.

In [8]:
m  = sents[0]
displacy.render(m, style='dep')
nice_print(m)

text           pos_       dep_        
_______________________________________________________
Была           AUX        cop         
у              ADP        case        
богини         NOUN       ROOT        
Деметры        PROPN      appos       
дочь           NOUN       nsubj       
Персефона      PROPN      nmod        
.              PUNCT      punct       
_______________________________________________________


Как видим, корневое слово, от которого идут все стелочки имеет метку ROOT при вызове функции dep_ к токену. Соответственно это можно использовать для построения вопроса.

Следующая сложность, это определится какой именно вопрос мы хотим задать к предложению.

## <b>Начало экспериментов</b>
#### <b>Предложение 1</b>

<b>'Была у богини Деметры дочь Персефона"</b>

Вопросы:
1. Как звали дочь богини Деметры? Кто был у богини Деметры дочерью? Была ли у богини Деметры дочь? 2. Кто был у богини Деметры?
2. Кто такие Деметра и Персефона? Кто такая Деметра? Кто такая Персефона? Что связывает Деметру и Персефону?
___

1. Рассмотрим тип вопроса "Кто такая " + <именнованная сущность>? Проблема будет только в слове "такой/такая". Однако ее в принципе можно решить, допустим, заведя отдельно массивы мужских и женских имен. Получим их при помощи парсинга данных parser.ipynb.

In [9]:
female_name = ['Авга', 'Авра', 'Автоноя', 'Агава', 'Агамеда', 'Агамеда', 'Аглавра', 'Аглая', 'Адрастея', 'Айтилла', 'Акакаллида', 'Алкестида', 'Алкидика', 'Алкимеда', 'Алкиона', 'Алкиона', 'Алкиониды', 'Алкиппа', 'Алкиппа', 'Алкмена', 'Алопа', 'Алфесибея', 'Алфея', 'Амфитея', 'Анаксарета', 'Анаксо', 'Андромаха', 'Андромеда', 'Антеида', 'Антемона', 'Антигона', 'Антигона', 'Антигона', 'Антиклея', 'Антиноя', 'Антиопа', 'Апемосина', 'Арахна', 'Арганфона', 'Аргия', 'Арета', 'Ариадна', 'Арна', 'Арсиноя', 'Арсиноя', 'Астидамия', 'Астидамия', 'Астимедуса', 'Астиоха', 'Астипалея', 'Аталанта', 'Аттида', 'Аэдона', 'Аэропа', 'Батия', 'Баубо', 'Бентесикима', 'Библида', 'БрисеидаГ', 'Галанфида', 'Галия', 'Гарпалика', 'Гарпалика', 'Гекала', 'Геката', 'Гекуба', 'Гелла', 'Гемифея', 'Гермиона', 'Геро', 'Гера', 'Герса', 'Гея', 'Гесиона', 'Гестия', 'Гефестина', 'Гилайера', 'Гималия', 'Гиппа', 'Гипподамия', 'Гипподамия', 'Гипсипила', 'Главка', 'Горга', 'Горгоны', 'Горгофона', 'Даная', 'Деидамия', 'Деипила', 'Дексифея', 'Деметра', 'Демоника', 'Деянира', 'Диогения', 'Дирка', 'Дриопа', 'Евадна', 'Евбея', 'Еврианасса', 'Евфросина', 'Елена', 'Ианта', 'Идея', 'Илиона', 'Ино', 'Ио', 'Иодама', 'Иокаста', 'Иола', 'Ипполита', 'Исмена', 'Ифианасса', 'Ифигения', 'Ифимедея', 'Калика', 'Каллидика', 'Каллисто', 'Канака', 'Кассандра', 'Кассиопея', 'Каталог', 'Керкира', 'Кидиппа', 'Килла', 'Кирена', 'Клеопатра', 'Клеохария', 'Климена', 'Климена', 'Клитемнестра', 'Клития', 'Клития', 'Коронида', 'Корониды', 'Краная', 'Креуса', 'Креуса', 'Ктесилла', 'Ктимена', 'Ланасса', 'Лаодамия', 'Лаодика', 'Левкиппа', 'Левкофея', 'Леда', 'Ливия', 'Лисимаха', 'Литея', 'Локрийские', 'Макария', 'Макело', 'Манто', 'Марпесса', 'Мегара', 'Медея',  'Меланфо', 'Мелеагриды', 'Мелибея', 'Мелисса', 'Мелита', 'Мемфида', 'Меропа', 'Меропа', 'Мессена', 'Метанира', 'Метопа', 'Миниады', 'Мирмека', 'Мирра', 'Мнестра', 'Молиона', 'Молпадия', 'Мэра', 'Навпрестиды', 'Навсикая', 'Немезида', 'Никиппа', 'Никтимена', 'Ниоба', 'Ниоба', 'Окироя', 'Омфала', 'Опида', 'Орифия', 'Орсеида', 'Ортея', 'Паллена', 'Пандора', 'Пандроса', 'Панкратида', 'Парфенос', 'Пасифая', 'Педиада', 'Пелопия', 'Пенелопа', 'Пердика', 'Перибея', 'Перибея', 'Перибея', 'Перигуна', 'Перимеда', 'Перимела', 'Перо', 'Персефона', 'Пирена', 'Пирра', 'Писидика', 'Полидора', 'Поликаста', 'Поликаста', 'Поликсена', 'Поликсо', 'Полифонта', 'Помфолига', 'Праксифея', 'Прокна', 'Прокрида', 'Проноя', 'Пропетиды', 'Протогенея', 'Протогенея', 'Протогония', 'Псамафа', 'Рео', 'Рода', 'Родопа', 'Рома', 'Саламина', 'Семела', 'Сивиллы', 'Сидеро', 'Синопа', 'Скилла', 'Спарта', 'Спутники', 'Стилба', 'СфенебеяТ', 'Талия', 'Текмесса', 'Телета', 'Телефасса', 'Тимандра', 'Тироф', 'Фантасия', 'Феано', 'Феано', 'Феба', 'Федра', 'Фемисто', 'Фемоноя', 'Феофана', 'Фива', 'Фива', 'Филемон', 'Филлида', 'Филомела', 'ФилонояХ', 'Халкиопа', 'Харикло', 'Хиона', 'Хиона', 'Хлорида', 'Хриса', 'Хрисеида', 'Хрисофемида', 'Хрисофемида', 'Хтония', 'Хтонофилаэ', 'Цирцея', 'Эвриала', 'Эвридика', 'Эвриклея', 'Эгиала', 'Эгиалея', 'Эгла', 'Эглеида', 'Элара', 'Электра', 'Электриона', 'Энарета', 'Эндеида', 'Энотрофы', 'Эпикаста', 'Эригона', 'Эригона', 'Эринома', 'Эрифила', 'Эфрая', 'Ямба']
female_name = set(female_name)

In [10]:
male_name = ['Абант', 'Абарис', 'Абдер', 'Авгий', 'Авсон', 'Автесион', 'Автолик', 'Автомедонт', 'Автоной', 'Агамед', 'Агамемнон', 'Агапенор', 'Агассамен', 'Агафирс', 'Агелай', 'Агенор', 'Агрий', 'Агрол', 'Агрон', 'Адмет', 'Адраст', 'Адраст', 'Аероп', 'Азан', 'Аид', 'Академ', 'Акамант', 'Акамант', 'Акамант', 'Акарнан', 'Акаст', 'Акербант', 'Акест', 'Акет', 'Акид', 'Аконтий', 'Акрисий', 'Актей', 'Актеон', 'Актий', 'Актор', 'Актор', 'Актор', 'Алалкомен', 'Алей', 'Алектрион', 'Алет', 'Алет', 'Алкандр', 'Алкафой', 'Алкафой', 'Алкей', 'Алкиды', 'Алкимаха', 'Алкимедонт', 'Алкиной', 'Алкиноя', 'Алкмеон', 'Алкмеон', 'Алкон', 'Алм', 'Алоады', 'Алоей', 'Алоей', 'Алфемен', 'Алфеп', 'Амаринкей', 'Амик', 'Амикл', 'Аминтор', 'Амисодар', 'Амифаон', 'Ампел', 'Амфиал', 'Амфиарай', 'Амфиктион', 'Амфилох', 'Амфилох', 'Амфимах', 'Амфимах', 'Амфином', 'Амфинома', 'Амфион', 'Амфион', 'Амфитрион', 'Амфотер', 'Анаксагор', 'Андрей', 'Андремон', 'Андрогей', 'Андрокл', 'Андропомп', 'Андрос', 'Аний', 'Анит', 'Анкей', 'Анкей', 'Антагор', 'Антей', 'Антенор', 'Антикл', 'Антилох', 'Антимах', 'Антиной', 'Антион', 'Антиох', 'Антиф', 'Антиф', 'Антифат', 'Анф', 'Анфей', 'Анфином', 'Анхиал', 'Анхироя', 'Анхис', 'Апис', 'Апсирт', 'Араб', 'Арант', 'Аргал', 'Аргос', 'Аргос', 'Ардал', 'Арейфой', 'Арисба', 'Аристей', 'Аркад', 'Аркесилай', 'Аркисий', 'Архелай', 'Архелох', 'Архител', 'Асий', 'Аскалаф', 'Аскалаф', 'Асканий', 'Ассарак', 'Астерий', 'Астерион', 'Астеродия', 'Астеропей', 'Астианакт', 'Астикратия', 'Астинома', 'Атимний', 'Атрей', 'Аттис', 'Афамант', 'Афарей', 'Афаретиды', 'Афидант', 'Афидант', 'Афидн', 'Ахей', 'Ахилл', 'Ацис', 'Аэтий', 'Аэфлий', 'Аякс', 'Аякс', 'Бакис', 'Батт', 'Бел', 'Беллерофонт', 'Беот', 'Биант', 'Бисса', 'Бореады', 'Борисфен', 'Бранх', 'Бранх', 'Бресия', 'Бронт', 'Бротей', 'Бузиг', 'Бусирис', 'Бут', 'Бут', 'Бут', 'Бут', 'Визант', 'ВирбийГ', 'Габид', 'Галат', 'Галиррофий', 'Галиферс', 'Ганимед', 'Гекаэрг', 'Гектор', 'Геланор', 'Гелен', 'Геликаон', 'Гелон', 'Гем', 'Гемон', 'Гениоха', 'Герана', 'Гермафродит', 'Гермион', 'Гиант', 'Гиацинт', 'Гиацинт', 'Гилас', 'Гилей', 'Гименей', 'Гиперет', 'Гипподамант', 'Гиппокоонт', 'Гиппокорист', 'Гипполох', 'Гипполох', 'Гиппомедонт', 'Гиппоной', 'Гиппот', 'Гиппофой', 'Гиппофой', 'Гиппофоонт', 'Гиппофоя', 'Гипсей', 'Гипсенор', 'Гирией', 'Гиртак', 'Главк', 'Главк', 'Главк', 'Гоплей', 'Гоплет', 'Горгира', 'Горгофон', 'Гордий', 'ГунейД', 'Давн', 'Даифрон', 'Дайт', 'Дамастор', 'Данай', 'Дардан', 'Дарет', 'Дафнис', 'Девкалион', 'Девкалион', 'Дедал', 'Дедалион', 'Деимах', 'Деион', 'Деипил', 'Деифоб', 'Деифонт', 'Дексамен', 'Дексикреон', 'Дельф', 'Дельфийская', 'Демодок', 'Демодок', 'Демолеонт', 'Демофонт', 'Демофонт', 'Демофонт', 'Десмонт', 'Диант', 'Диктис', 'Диктис', 'Димант', 'Димет', 'Диокл', 'Диоксиппа', 'Диомед', 'Диомед', 'Дион', 'Дисавл', 'Долий', 'Долих', 'Долон', 'Дор', 'ДриопЕ', 'Евдора', 'Евемон', 'Евипп', 'Евмед', 'Евмей', 'Евмел', 'Евмел', 'Евмолп', 'Евней', 'Евном', 'Евпиф', 'Евриал', 'Еврибий', 'Евриганея', 'Евридамант', 'Еврилох', 'Евримах', 'Еврипил', 'Еврипил', 'Еврипил', 'Еврипил', 'Еврисак', 'Еврисфей', 'Еврит', 'Еврит', 'Евритион', 'Евритион', 'Евритион', 'Евритион', 'Еврифемида', 'Еврот', 'Евфем', 'Евфет', 'ЕвфорбЖ', 'Женихи', 'Женихи', 'Зарекс', 'Зефи', 'Зевс', 'Иалмен', 'Иам', 'Ианира', 'Ианфа', 'Иарбант', 'Иас', 'Иасий', 'Иасион', 'Идас', 'Идмон', 'Идомена', 'Идоменей', 'Икадий', 'Икар', 'Икарий', 'Икарий', 'Иксион', 'Ил', 'Иллирий', 'Имбрас', 'Иммарад', 'Иобат', 'Иокаст', 'Иолай', 'Ион', 'Ипполит', 'Ир', 'Ир', 'Ис', 'Исхен', 'Исхий', 'Итил', 'Итимоней', 'Итис', 'Итон', 'Ификл', 'Ификл', 'Ификл', 'Ифиноя', 'Ифис', 'Ифис', 'Ифит', 'Ифит', 'Каанф', 'Кавкон', 'Кавн', 'Кадм', 'Каламос', 'Калидон', 'Калхас', 'Кандал', 'Канеф', 'Каноп', 'Канф', 'Капаней', 'Капис', 'Кар', 'Кар', 'Каран', 'Карней', 'Катрей', 'Кебрион', 'Кеик', 'Кронос', 'Крон', 'Кекропс', 'Кекропс', 'Келей', 'Кельт', 'Кеней', 'Керамб', 'Керик', 'Керкаф', 'Керкион', 'Кефал', 'Кефал', 'Кефей', 'Кефей', 'Кианипп', 'Кидон', 'Кизик', 'Кикн', 'Кикн', 'Кикн', 'Кикн', 'Килик', 'Кинир', 'Кинорт', 'Кипарис', 'Кипсел', 'Киссей', 'Китисор', 'Киферон', 'Кихрей', 'Кларос', 'Клейто', 'Клеобея', 'Клеобула', 'Клеодора', 'Клеомед', 'Клеотера', 'Климен', 'Климен', 'Климен', 'Клинис', 'Клиситира', 'Клита', 'Клитий', 'Клоний', 'Кноп', 'Кодр', 'Кокал', 'Колен', 'Комето', 'Копрей', 'Кореб', 'Кореб', 'Корибант', 'Коринн', 'Коринф', 'Кориф', 'Корон', 'Крагалей', 'Кранай', 'Крантор', 'Креонт', 'Креонт', 'Крефей', 'Крокон', 'Крокус', 'Кротон', 'Кротоп', 'Кротос', 'Ксанф', 'Ксанф', 'Ксуф', 'КтесиппЛ', 'Лаий', 'Лакедемон', 'Лакестад', 'Лакий', 'Ламедон', 'Лампетия', 'Лаогора', 'Лаодамант', 'Лаодик', 'Лаокоонт', 'Лаомед', 'Лаомедонт', 'Лаонома', 'Лаофоя', 'Лапиф', 'Латин', 'Лаэрт', 'Леарх', 'Левк', 'Левкипп', 'Левкипп', 'Левкипп', 'Левкипп', 'Левкоком', 'Леит', 'Лелег', 'Леодамант', 'Леонт', 'Леонтей', 'Лепрей', 'Лид', 'Лик', 'Лик', 'Лик', 'Лик', 'Лик', 'Ликаон', 'Ликаон', 'Ликимний', 'Ликомед', 'Ликомед', 'Ликург', 'Ликург', 'Ликург', 'Лин', 'Линкей', 'Линкей', 'Линкей', 'Липар', 'Лирк', 'Лисиппа', 'Лиситея', 'Литиерс', 'Лихас', 'ЛокрМ', 'Магнет', 'Макарей', 'Макарей', 'Македон', 'Мантий', 'Марафон', 'Марон', 'Маттан', 'Махаон', 'Мегапенф', 'Мегапенф', 'Мегарей', 'Мегет', 'Медей', 'Медонт', 'Медонт', 'Мекистей', 'Мекон', 'Мелампод', 'Мелан', 'Меланей', 'Меланион', 'Меланипп', 'Меланипп', 'Меланф', 'Меланфий', 'Мелеагр', 'Меликерт', 'Мелисс', 'Мелос', 'Мемнон', 'Менекей', 'Менекей', 'Менелай', 'Менесфей', 'Менетий', 'Менит', 'Ментор', 'Меон', 'Мерион', 'Мермер', 'Мероп', 'Меропида', 'Местор', 'Местор', 'Местор', 'Метапонт', 'Метион', 'Мигдон', 'Мигдон', 'Мидас', 'Милет', 'Милет', 'Милих', 'Миний', 'Минос', 'Мирина', 'Мирмидон', 'Миртил', 'Мисен', 'Молиониды', 'Молорх', 'Молосс', 'Мопс', 'Мопс', 'Мопс', 'Муних', 'МусейН', 'Навплий', 'Навплий', 'Навплий', 'Навсифой', 'Навсифой', 'Навсифой', 'Наннак', 'Нарцисс', 'Нелеиды', 'Нелей', 'Нелей', 'Неоптолем', 'Нестор', 'Никострат', 'Никострата', 'Никтим', 'Нин', 'Нирей', 'Нис', 'Нис', 'НораксО', 'Обрим', 'Огиг', 'Одиссей', 'Оилей', 'Окн', 'Окридион', 'Оксил', 'Оксиний', 'Оксинт', 'Олен', 'Олимп', 'Олинф', 'Онк', 'Опунт', 'Орест', 'Оресфей', 'Орион', 'Ормен', 'Орней', 'Орседика', 'Орсилох', 'Орфей', 'Орхомен', 'Орхомен', 'Отрей', 'Офельт', 'Офрейда', 'ОхимП', 'Паламед', 'Палемоний', 'Палефат', 'Паллант', 'Паллантиды', 'Памфил', 'Панасагор', 'Пандар', 'Пандарей', 'Пандион', 'Пандион', 'Пандион', 'Пандор', 'Панопей', 'Панф', 'Парис', 'Парфенопей', 'Патрокл', 'Пафос', 'Пеант', 'Пелагонт', 'Пеласг', 'Пеласг', 'Пеласг', 'Пелей', 'Пелий', 'Пелоп', 'Пемандр', 'Пенелей', 'Пенфил', 'Пергам', 'Периер', 'Периклимен', 'Периклимен', 'Перилай', 'Перифант', 'Перифет', 'Перс', 'Перс', 'Персей', 'Персепол', 'Петеой', 'Пигмалион', 'Пигмалион', 'Пиер', 'Пилад', 'Пилас', 'Пилемен', 'Пилия', 'Пирант', 'Пирехм', 'Пирифой', 'Писенор', 'Писистрат', 'Питфей', 'Плеврон', 'Плексипп', 'Плексипп', 'Плисфен', 'Плисфен', 'Подалирий', 'Подарк', 'Полиб', 'Полиб', 'Полидамант', 'Полидамна', 'Полидект', 'Полидор', 'Полиид', 'Поликаон', 'Поликсен', 'Полимнестор', 'Полиник', 'Полипет', 'Полит', 'Полит', 'Полифем', 'Полифонт', 'Поллид', 'Полтис', 'Порфаон', 'Превген', 'Прет', 'Приам', 'Прокл', 'Прокруст', 'Промах', 'Промет', 'Просимн', 'Протесилай', 'Профоенор', 'Профой', 'Псилл', 'ПтерелайР', 'Радамант', 'Ракий', 'Рар', 'Рес', 'Салмоней', 'Саон', 'Сард', 'Сарданапал', 'Сарон', 'Сарпедон', 'Сарпедон', 'Сизиф', 'Сикел', 'Сикион', 'Силей', 'Синис', 'Синон', 'Сифон', 'Скир', 'Скирон', 'Смикр', 'Смилакс', 'Спарты', 'Стафил', 'Стентор', 'Стимфал', 'Строфий', 'Суфакс', 'Сфенел', 'Сфенел', 'Сфенела', 'Схедий', 'СхенейТ', 'Талай', 'Талос', 'Талфибий', 'Тантал', 'Тантал', 'Тарент', 'Тевкр', 'Тевкр', 'Тевтис', 'Тевфрант', 'Тегирий', 'Тектам', 'Теламон', 'Телегон', 'Телегон', 'Телемах', 'Телеф', 'Тенаг', 'Тенес', 'Теоклимен', 'Терей', 'Терсандр', 'Терсит', 'Тесей', 'Тидей', 'Тингис', 'Тиндарей', 'Тиресий', 'Тиррен', 'Тисамен', 'Тисамен', 'Титий', 'Титий', 'Титон', 'Тифий', 'Тлеполем', 'Тмол', 'Тон', 'Трамбел', 'Триоп', 'Триоп', 'Триоп', 'Триптолем', 'Троил', 'Трос', 'ТрофонийФ', 'Фаланг', 'Фалер', 'Фалпий', 'Фамирид', 'Фаон', 'Фасос', 'Фаэтон', 'Фаэтуса', 'Фегей', 'Фемий','Феникс', 'Фенодамант', 'Феодамант', 'Ферас', 'Ферекл', 'Ферет', 'Феспий', 'Фессал', 'Фессал', 'Фестий', 'Фестор', 'Фидипп', 'Фиест', 'Филак', 'Филаммон', 'Филант','Филей', 'Филетий', 'Филоктет', 'Филомел', 'Филонома', 'Филот', 'Фимет', 'Финей', 'Финей', 'Фитал', 'Флегий', 'Флиант', 'Фоант', 'Фок', 'Фоон', 'Форбант', 'Форбант', 'Форбант', 'Форкий', 'Фороней', 'Фрасий', 'Фрасимед', 'Фрикс', 'Фтий', 'Халкин', 'Халкодонт', 'Халкодонт', 'Хаон', 'Хароп', 'Херон', 'Хорик', 'Хрис', 'Хрис', 'Хрисипп', 'Эак', 'Эакиды', 'Эбал', 'Эвандр', 'Эвандр', 'Эвбулей', 'Эвен', 'Эвиппа', 'Эвпалам', 'Эвриал', 'Эврибат', 'Эвримедон', 'Эвримедон', 'Эвримедон', 'Эгей', 'Эгиалей', 'Эгиалей', 'Эгимий', 'Эгипт', 'Эгиптиады', 'Эгиптий', 'Эгисф', 'Эионей', 'Экл', 'Элат', 'Элат', 'Элат', 'Элатон', 'Элевсин', 'Элевтер', 'Элей', 'Электрион', 'Элефенор', 'Элим', 'Эллин', 'Элпенор', 'Энал', 'Эндимион', 'Эндой', 'Эней', 'Эномай', 'Энопион', 'Энотр', 'Эол', 'Эон', 'Эпаф', 'Эпей', 'Эпей', 'Эпистроф', 'Эпит', 'Эпит', 'Эпопей', 'Эргин', 'Эргин', 'Эрехтей', 'Эрибот', 'Эрикс', 'Эрисихтон', 'Эрисихтон', 'Эрифия', 'Эрифра', 'Эрихтоний', 'Эсак', 'Эсон', 'Этеокл', 'Этол', 'Эфалид', 'Эхем', 'Эхет', 'Эхетл', 'Эхион', 'Ээт', 'Ээтионя', 'Япиг', 'Ясон']
male_name = set(male_name)

In [11]:
# тип вопроса "Кто такой ... " + <именнованная сущность>?
def who_is(doc):
    arr = []
    for token in doc.ents:
        if token.label_ == "PER":
            flag_1 = [elem for elem in female_name if elem.find(token.text[:-1]) != -1]
            flag_2 = [elem for elem in male_name if elem.find(token.text[:-1]) != -1]
            if flag_1:
                arr.append("Кто такая " + flag_1[0] + "?")
            elif flag_2:
                arr.append("Кто такой " + flag_2[0] + "?")
            else:
                arr.append("Кто такое " + token.text + "?")
    return arr

def go(sents, func):
    info = []
    for sent in sents:
        info.append(func(sent))
    return info

print(who_is(sents[0]))
print(go(sents, who_is))

[['Кто такая Деметра?', 'Кто такая Персефона?'], ['Кто такой Кронос?', 'Кто такой Зевс?'], ['Кто такая Персефона?'], ['Кто такая Деметра?'], [], ['Кто такая Персефона?', 'Кто такой Зевс?'], ['Кто такая Персефона?'], ['Кто такой Зевс?', 'Кто такой Аид?', 'Кто такая Персефона?']]
['Кто такая Деметра?', 'Кто такая Персефона?']


2. Теперь попробуем поработать с деревом зависимостей и паттернами. Для того чтобы задать вопрос от корня ROOT. 

In [12]:
case = {'была': "был"}
key = case.keys()

def question_1(doc, left, right):
    phrase = ''
    for token in doc:
        if token.dep_.find('ROOT') != -1:
            if phrase.find(token.text) == -1:
                phrase = phrase + ' ' + token.text
            while True:
                if token.head and phrase.find(token.head.text) == -1:
                    phrase = phrase + ' ' + token.head.text

                if list(token.lefts) != [] and left:
                    arr = [str(elem) for elem in list(token.lefts)]
                    phrase = create_str_on_arr(arr) + phrase

                if list(token.rights) != [] and right:
                    arr = [str(elem) for elem in list(token.rights)]
                    phrase = phrase + ' ' + arr[0]
                break

    phrase = phrase.replace("  ", " ").replace(" .", ".").strip()
    for token in doc:
        if token.lemma_ in key:
            count = len(token)
            phrase = "Кто " + case[token.lemma_] + " " + phrase[count+1:]

    phrase += "?"
        
    return phrase

def go_1(sents, left, right):
    info = []
    for sent in sents:
        info.append(question_1(sent, left, right))
    return info

def create_str_on_arr(arr):
    phrase = ' '
    for elem in arr:
        phrase += elem + ' '
    return phrase

print(question_1(sents[0], True, True))
print(go_1(sents, True, True))
displacy.render(sents[0], style='dep')

Кто был у богини Деметры?
['Кто был у богини Деметры?', 'Отцом Персефоны?', 'Однажды Персефона вместе беззаботно резвилась долине?', 'бабочке перебегала дочь?', 'Она рвала розы?', 'Беспечно резвилась Персефона?', 'Не думала Персефона?', 'Кто был отдал ее?']


#### <b>Предложение 2</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Отцом Персефоны был сын Крона, громовержец Зевc.'</b>

Вопросы:
1. Кто такая Персефона? Кто такой Крон? Кто такой Зевс?
2. Кто был отцом Персефоны?
3. Чьим сыном был громовержец Зевс?

___

1. Как видно, вопросы типа 1 мы уже можем сформировывать.

In [13]:
print(who_is(sents[1]))

['Кто такой Кронос?', 'Кто такой Зевс?']


In [14]:
def nice_print(doc):
    print('{:<14} {:<10} {:<12}'.format('text', 'pos_', 'dep_'))
    print('_'* 55)
    for token in doc:
        print('{:<14} {:<10} {:<12}'.format(token.text, token.pos_, token.dep_))
    print('_'* 55)

2. Попробуем задать вопрос "Кто был " + <паттерн>?
В нашел случае необходмо получить паттерн, которому будет отвечать фраза "отцом Персефоны".

In [15]:
nice_print(sents[1])

text           pos_       dep_        
_______________________________________________________
Отцом          NOUN       ROOT        
Персефоны      PROPN      nmod        
был            AUX        cop         
сын            NOUN       nsubj       
Крона          PROPN      nmod        
,              PUNCT      punct       
громовержец    NOUN       appos       
Зевс           PROPN      appos       
.              PUNCT      punct       
_______________________________________________________


Как видим, необходимым нам словам соответствуют части речи -- NOUN и PROPN, а их метки зависимостей -- ROOT и nmod. С помощью методов spacy можно легко просмотреть трактовку вышеупомянутых меток, то есть для слова "Персефона" мы получаем *имя собственное* и *модификатор номинального*. А для слова "отцом" - *имя существительное* и *корень*.

In [16]:
print("{:<10} {:<14} {:<15}".format(sents[1][0].text, sp.explain(sents[1][0].pos_), sp.explain(sents[1][0].dep_)))
print("{:<10} {:<14} {:<15}".format(sents[1][1].text, sp.explain(sents[1][1].pos_), sp.explain(sents[1][1].dep_)))


Отцом      noun           root           
Персефоны  proper noun    modifier of nominal


Так и осуществим поиск нашего паттерна.

In [17]:
def pattern_1(doc):
    for i in range(len(doc)-1):
        if doc[i].pos_ == 'NOUN' and doc[i+1].pos_ == 'PROPN':
            return [doc[i], doc[i+1]]
    return False

def question_2(doc):
    ans = "Кто был "
    arr = pattern_1(doc)
    if arr:
        for elem in arr:
            ans += elem.text + " "
        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None

print(go(sents, question_2))
print(question_2(sents[1]))

['Кто был богини Деметры?', 'Кто был Отцом Персефоны?', None, 'Кто был дочь Деметры?', None, None, None, 'Кто был брату Аиду?']
Кто был Отцом Персефоны?


№3. Теперь попробуем сформулировать вопрос "Чьим сыном был громовержец Зевс?"

In [18]:
nice_print(sents[1])
displacy.render(sents[1], style='dep')

text           pos_       dep_        
_______________________________________________________
Отцом          NOUN       ROOT        
Персефоны      PROPN      nmod        
был            AUX        cop         
сын            NOUN       nsubj       
Крона          PROPN      nmod        
,              PUNCT      punct       
громовержец    NOUN       appos       
Зевс           PROPN      appos       
.              PUNCT      punct       
_______________________________________________________


Нас интересуют следующие слова: 

| сын  | NOUN | nsubj |
|  :---:  |  :----:  |  :---:  |
| громовержец    | NOUN   | appos |
| Зевс    | PROPN | appos | 

In [19]:
def pattern_3(doc):
    for i in range(len(doc)-1):      
        if doc[i].pos_ == 'NOUN' and doc[i].dep_ == 'nsubj' and doc[i].children:
            res = [] 
            res.append(doc[i])
            arr = list(doc[i].children)
            for i in range(len(arr)):
                if arr[i].pos_ == "NOUN" and arr[i].dep_ == 'appos':
                    res.append(arr[i])
                    if arr[i].children:
                        arr_2 = list(arr[i].children)
                        res += [elem for elem in arr_2 if (elem.pos_ == 'PROPN')] 
            return res
    return False

def question_3(doc):
    ans = "Чьим "
    arr = pattern_3(doc)
    if arr:
        i = 0
        for elem in arr:
            if i == 0:
                ans += elem.text.lower() + " был "
                i = 1
            else:
                ans += elem.text + " "
        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None

print(go(sents, question_3))

['Чьим дочь был?', 'Чьим сын был громовержец Зевс?', None, 'Чьим дочь был?', None, 'Чьим отец был?', None, 'Чьим света был?']


Результат, который мы получили такой - 'Чьим сын был громовержец Зевс?'. Алгоритм построения сработал верно, однако, проблема с окончаниями здесь решается только вручную.

#### <b>Предложение 3</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Однажды Персефона вместе со своими подругами, океанидами, беззаботно резвилась в цветущей Нисейской долине.'</b>

Вопросы:
1. Кто такая Персефона?
2. Что делала Персефона в цветущей Нисейской долине?
3. Где однажды резвилась Персефона?



**№ 1**. Как видно, вопросы типа 1 мы уже можем сформировывать.

In [20]:
print(who_is(sents[2]))

['Кто такая Персефона?']


**№ 2**. Попробуем задать вопрос - "Что делала Персефона в цветущей Нисейской долине?". Для того чтобы этот вопрос был задан, необходимо наличие подлежащего, совершающего действие в каком-то определенном месте.


In [21]:
displacy.render(sents[2], style='dep')

In [22]:
nice_print(sents[2])

text           pos_       dep_        
_______________________________________________________
Однажды        ADV        advmod      
Персефона      PROPN      nsubj       
вместе         ADV        advmod      
со             ADP        case        
своими         DET        det         
подругами      NOUN       obl         
,              PUNCT      punct       
океанидами     NOUN       conj        
,              PUNCT      punct       
беззаботно     ADV        advmod      
резвилась      VERB       ROOT        
в              ADP        case        
цветущей       VERB       amod        
Нисейской      ADJ        amod        
долине         NOUN       obl         
.              PUNCT      punct       
_______________________________________________________


In [23]:
print(sp.explain(sents[2][14].dep_))

oblique nominal


In [24]:
def pattern_4(doc):
    for i in range(len(doc)): 
        if doc[i].dep_ == 'ROOT' and doc[i].children:
            res = []
            person = get_token(doc, list(doc[i].children), 'PER')
            location = get_token(doc, list(doc[i].children), 'LOC')
            if person and location:
                res.append(person)
                if location.children:
                    res += list(location.children)
                res.append(location)
                return res
    return False


def get_token(doc, arr, ent):
    for i in range(len(doc)):
        for j in range(len(arr)):
            if doc[i] == arr[j] and doc[i].ent_type_ == ent:
               return doc[i]       
    return False

def question_4(doc):
    ans = "Что "
    arr = pattern_4(doc)
    i = 0
    if arr:
        for token in arr:
            if i == 0:
                flag_1 = [elem for elem in female_name if elem.find(token.text[:-1]) != -1]
                flag_2 = [elem for elem in male_name if elem.find(token.text[:-1]) != -1]

                if flag_1:
                    ans += " делала " + flag_1[0] 
                elif flag_2:
                    ans += " делал " + flag_2[0]
                else:
                    ans += " делал " + token.text

                i += 1
            else:
                ans += " " + token.text


        return ans.strip() + "?"
    else:
        return None
    
print(go(sents, question_4))

[None, None, 'Что  делала Персефона в цветущей Нисейской долине?', None, None, None, None, None]


**№ 3**. Попробуем задать вопрос - "Где однажды резвилась Персефона?". Для того чтобы этот вопрос был задан, также  необходимо наличие подлежащего, совершающего действие в каком-то определенном месте. Только в этом случае мы будем задавать вопросы к месту.

In [25]:
displacy.render(sents[2], style='dep')
nice_print(sents[2])

text           pos_       dep_        
_______________________________________________________
Однажды        ADV        advmod      
Персефона      PROPN      nsubj       
вместе         ADV        advmod      
со             ADP        case        
своими         DET        det         
подругами      NOUN       obl         
,              PUNCT      punct       
океанидами     NOUN       conj        
,              PUNCT      punct       
беззаботно     ADV        advmod      
резвилась      VERB       ROOT        
в              ADP        case        
цветущей       VERB       amod        
Нисейской      ADJ        amod        
долине         NOUN       obl         
.              PUNCT      punct       
_______________________________________________________


In [26]:
def pattern_5(doc):
    for i in range(len(doc)): 
        if doc[i].dep_ == 'ROOT' and doc[i].children:
            res = []
            person = get_token(doc, list(doc[i].children), 'PER')
            location = get_token(doc, list(doc[i].children), 'LOC')
            advmod = get_token_2(doc, list(doc[i].children), 'advmod')
            if person and location:
                if advmod:
                    res.append(advmod)
                res.append(doc[i])
                res.append(person)
                return res
    return False

def get_token_2(doc, arr, dep):
    for i in range(len(doc)):
        for j in range(len(arr)):
            if doc[i] == arr[j] and doc[i].dep_ == dep:
               return doc[i]       
    return False

def question_5(doc):
    ans = "Где"
    arr = pattern_5(doc)
    if arr:
        for token in arr:
            flag_1 = [elem for elem in female_name if elem.find(token.text[:-1]) != -1]
            flag_2 = [elem for elem in male_name if elem.find(token.text[:-1]) != -1]
            if flag_1 or flag_2:
                ans += " " + token.text 
            else:
                ans += " " + token.text.lower()
            flag_1, flag_2 = False, False

        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None
    
print(go(sents, question_5))
print(question_5(sents[2]))

[None, None, 'Где однажды резвилась Персефона?', None, None, None, None, None]
Где однажды резвилась Персефона?


#### <b>Предложение 4</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Подобно бабочке перебегала дочь Деметры от цветка к цветку.'</b>

Вопросы:
1. Кто такая Деметра?
2. Подобному кому перебегала дочь Деметры от цветка к цветку?


**№ 1**. Как видно, вопросы типа 1 мы уже можем сформировывать.

In [27]:
print(sents[3])
print(who_is(sents[3]))


Подобно бабочке перебегала дочь Деметры от цветка к цветку.
['Кто такая Деметра?']


**№ 2**. Попробуем задать вопросов номер 2: "Подобному кому перебегала дочь Деметры от цветка к цветку?"

In [28]:
displacy.render(sents[3], style='dep')
nice_print(nlp("Подобно бабочке перебегала дочь Деметры от цветка к цветку."))

text           pos_       dep_        
_______________________________________________________
Подобно        ADP        case        
бабочке        NOUN       obl         
перебегала     VERB       ROOT        
дочь           NOUN       nsubj       
Деметры        PROPN      appos       
от             ADP        case        
цветка         NOUN       obl         
к              ADP        case        
цветку         NOUN       nmod        
.              PUNCT      punct       
_______________________________________________________


In [29]:
displacy.render(sents[3], style = 'ent')

In [30]:
def pattern_6(doc):
    for i in range(len(doc)): 
        if doc[i].dep_ == 'ROOT' and doc[i].children:
            res = []
            object = get_token_2(doc, list(doc[i].children), 'obl')       # слово с чем сравнивают
            noun = get_token_2(doc, list(doc[i].children), 'nsubj')
            if object and object.children:
                comparison = get_token_3(doc, list(object.children), 'ADP', 'case')
                if comparison:
                    res.append(doc[i])
                    if noun and noun.children:
                        res.append(noun)
                        noun_2 = get_token(doc, list(noun.children), 'PER')
                        res.append(noun_2)
                    return res
    return False

def get_token_3(doc, arr, pos, dep):
    for i in range(len(doc)):
        for j in range(len(arr)):
            if doc[i] == arr[j] and doc[i].dep_ == dep and doc[i].pos_ == pos:
               return doc[i]       
    return False

def question_6(doc):
    ans = "Подобно кому"
    arr = pattern_6(doc)
    if arr:
        for token in arr:
            if token:
                ans += " " + token.text 

        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None

print(go(sents, question_6))
print(question_6(sents[3]))

[None, None, 'Подобно кому резвилась Персефона?', 'Подобно кому перебегала дочь Деметры?', None, None, None, 'Подобно кому отдал Зевс?']
Подобно кому перебегала дочь Деметры?


#### <b>Предложение 5</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Она рвала розы, фиалки, лилии и гиацинты.'</b>

Вопросы:
1. Что она рвала?

In [31]:
displacy.render(sents[4], style='dep')
nice_print(sents[4])

text           pos_       dep_        
_______________________________________________________
Она            PRON       nsubj       
рвала          VERB       ROOT        
розы           NOUN       obj         
,              PUNCT      punct       
фиалки         NOUN       conj        
,              PUNCT      punct       
лилии          NOUN       conj        
и              CCONJ      cc          
гиацинты       NOUN       conj        
.              PUNCT      punct       
_______________________________________________________


In [32]:
def pattern_7(doc):
    for i in range(len(doc)): 
        if doc[i].dep_ == 'ROOT' and doc[i].children:
            res = []
            obj = get_token_2(doc, list(doc[i].children), 'obj')
            person = get_token_2(doc, list(doc[i].children), 'nsubj')
            if obj and person:
                res.append(person)
                res.append(doc[i])
                return res
    return False


def question_7(doc):
    ans = "Что"
    arr = pattern_7(doc)
    if arr:
        for token in arr:
            flag_1 = [elem for elem in female_name if elem.find(token.text) != -1]
            flag_2 = [elem for elem in male_name if elem.find(token.text) != -1]
            if flag_1:
                ans += " " + flag_1[0]
            elif flag_2:
                ans += " " + flag_2[0]
            else:
                ans += " " + token.text.lower()

        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None

print(go(sents, question_7))

[None, None, None, None, 'Что она рвала?', None, None, 'Что Зевс отдал?']


#### <b>Предложение 6</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Беспечно резвилась Персефона, не ведая той судьбы, которую назначил ей отец ее Зевс.'</b>

Вопросы:
1. Кто такая Персефона? Кто такой Зевс?
2. Что делала Персефона?

**№ 1**. Как видно, вопросы типа 1 мы уже можем сформировывать.

In [33]:
print(sents[5])
print(who_is(sents[5]))

Беспечно резвилась Персефона, не ведая той судьбы, которую назначил ей отец ее Зевс.
['Кто такая Персефона?', 'Кто такой Зевс?']


**№ 2**. Что делала Персефона?

In [34]:
nice_print(sents[5])

text           pos_       dep_        
_______________________________________________________
Беспечно       ADV        advmod      
резвилась      VERB       ROOT        
Персефона      PROPN      nsubj       
,              PUNCT      punct       
не             PART       advmod      
ведая          VERB       advcl       
той            DET        det         
судьбы         NOUN       obj         
,              PUNCT      punct       
которую        PRON       obj         
назначил       VERB       acl:relcl   
ей             PRON       iobj        
отец           NOUN       nsubj       
ее             DET        det         
Зевс           PROPN      nsubj       
.              PUNCT      punct       
_______________________________________________________


In [35]:
displacy.render(sents[5], style='dep')

In [36]:
def pattern_8(doc):
    for i in range(len(doc)): 
        if doc[i].dep_ == 'ROOT' and doc[i].children:
            nsubj = get_token(doc, list(doc[i].children), 'PER')
            if nsubj:
                return [nsubj]
    return False

def question_8(doc):
    ans = "Что"
    arr = pattern_8(doc)
    if arr:
        for token in arr:
            flag_1 = [elem for elem in female_name if elem.find(token.text[:-1]) != -1]
            flag_2 = [elem for elem in male_name if elem.find(token.text[:-1]) != -1]
            if flag_1:
                ans += " делала " + flag_1[0]
            elif flag_2:
                ans += " делал " + flag_2[0]
            else:
                ans += " делал " + token.text.lower()    

        return ans.strip() + "?"
    else:
        # отдаленные вопросы
        #return "Вы поняли суть мифа?"
        return None

print(go(sents, question_8))

['Что делала Деметра?', None, 'Что делала Персефона?', None, None, 'Что делала Персефона?', 'Что делала Персефона?', 'Что делал Зевс?']


#### <b>Предложение 7</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Не думала Персефона, что не скоро увидит она опять свет солнца, не скоро будет любоваться цветами и вдыхать их аромат.'</b>

Вопросы:
1. Кто такая Персефона?
2. Что делала Персефона?

**№ 1**. Как видно, вопросы типа 1 и 2 мы уже можем сформировывать.

In [37]:
print(sents[6])
print(who_is(sents[6]))
print(go(sents, question_8))

Не думала Персефона, что не скоро увидит она опять свет солнца, не скоро будет любоваться цветами и вдыхать их аромат.
['Кто такая Персефона?']
['Что делала Деметра?', None, 'Что делала Персефона?', None, None, 'Что делала Персефона?', 'Что делала Персефона?', 'Что делал Зевс?']


#### <b>Предложение 8</b>
Попробуем рассмотреть следующее предложение и создать функции, которые будут генерировать к нему вопросы.

<b>'Зевс отдал ее в жены своему брату Аиду, властителю царства теней умерших, и с ним была жить Персефона во мраке царства, лишенная света и солнца.'</b>

Вопросы:
1. Кто такая Персефона? Кто такой Зевс? Кто такой Аид?
2. Что сделал Зевс?

**№ 1**. Как видно, вопросы типа 1 и 2 мы уже можем сформировывать.

In [38]:
print(sents[7])
print(who_is(sents[7]))
print(go(sents, question_8))

Зевс отдал ее в жены своему брату Аиду, властителю царства теней умерших, и с ним была жить Персефона во мраке царства, лишенная света и солнца.
['Кто такой Зевс?', 'Кто такой Аид?', 'Кто такая Персефона?']
['Что делала Деметра?', None, 'Что делала Персефона?', None, None, 'Что делала Персефона?', 'Что делала Персефона?', 'Что делал Зевс?']


### Подведем итоги всех вопросов к этому абзацу.

In [39]:
def all_question(text):
    Dict = {}
    i = 1
    for sent in text:
        Dict[i] = []
        Dict[i] += who_is(sent)
        Dict[i].append(question_1(sent, True, True))
        Dict[i].append(question_2(sent))
        Dict[i].append(question_3(sent))
        Dict[i].append(question_4(sent))
        Dict[i].append(question_5(sent))
        Dict[i].append(question_6(sent))
        Dict[i].append(question_7(sent))
        Dict[i].append(question_8(sent))

        # удаление дубликатов и None
        Dict[i] = list(set(Dict[i]))
        if None in Dict[i]:
            Dict[i].remove(None)
        i += 1
    return Dict


# final = all_question(sents)

# for key, value in final.items():
#     print(key, ":", sents[key-1], "\n",value, "\n")


## Тестирование на новом кусочке текста

In [40]:
doc = nlp(u"Вдруг разверзлась земля, и на черных конях появился из земли в золотой колеснице владыка царства теней умерших, мрачный Аид. Он схватил юную Персефону, поднял ее на свою колесницу и в мгновение ока скрылся на своих быстрых конях в недрах земли. Только вскрикнуть успела Персефона. Далеко разнесся крик ужаса юной дочери Деметры; он донесся и до морских пучин, и до высокого, светлого Олимпа. Никто не видел, как похитил Персефону мрачный Аид, видел лишь его бог Гелиос-Солнце.Богиня Деметра услыхала крик Персефоны. Она поспешила в Нисейскую долину, всюду искала дочь; спрашивала подруг ее, океанид, но нигде не было ее. Океаниды не видали, куда скрылась Персефона.Тяжкая скорбь об утрате единственной возлюбленной дочери овладела сердцем Деметры. Одетая в темные одежды, девять дней, ничего не сознавая, ни о чем не думая, блуждала великая богиня Деметра по земле, проливая горькие слезы. Она всюду искала Персефону, всех просила о помощи, но никто не мог помочь ей в ее горе. Наконец, уже на десятый день она пришла к богу Гелиосу-Солнцу и стала со слезами молить его:- О, лучезарный Гелиос! Ты объезжаешь на златой колеснице высоко по небу всю землю и все моря, ты видишь все, ничто не может скрыться от тебя; если ты имеешь хоть немного жалости к несчастной матери, то скажи мне, где моя дочь Персефона, скажи, где мне искать ее! Я слышала ее крик, ее похитили у меня. Скажи, кто похитил ее. Я всюду искала ее, но нигде не могу найти! Ответил Деметре лучезарный Гелиос: - Великая богиня, ты знаешь, как я чту тебя, ты видишь, как скорблю, видя твое горе. Знай, великий тучегонитель Зевс отдал дочь твою в жены своему мрачному брату, владыке Аиду. Он похитил Персефону и увез ее в свое полное ужасов царство. Побори же свою тяжелую печаль, богиня; ведь велик муж твоей дочери, она стала женой могущественного брата великого Зевса. Еще больше опечалилась богиня Деметра. Разгневалась она на громовержца Зевса за то, что отдал он без ее согласия Персефону в жены Аиду. Она покинула богов, покинула светлый Олимп, приняла вид простой смертной и, облекшись в темные одежды, долго блуждала между смертными, проливая горькие слезы.")

sents_2 = list(doc.sents)
sents_2 = [delete_other(i) for i in sents_2]

for sent in sents_2:
    print(sent)

Вдруг разверзлась земля, и на конях появился из земли в колеснице владыка царства теней умерших, Аид.
Он схватил Персефону, поднял ее на свою колесницу и в мгновение ока скрылся на своих конях в недрах земли.
Только вскрикнуть успела Персефона.
Далеко разнесся крик ужаса дочери Деметры ; он донесся и до пучин, и до, Олимпа.
Никто не видел, как похитил Персефону Аид, видел лишь его бог Гелиос - Солнце.
Богиня Деметра услыхала крик Персефоны.
Она поспешила в Нисейскую долину, всюду искала дочь ; спрашивала подруг ее, океанид, но нигде не было ее.
Океаниды не видали, куда скрылась Персефона.
скорбь об утрате возлюбленной дочери овладела сердцем Деметры.
Одетая в одежды, девять дней, ничего не сознавая, ни о чем не думая, блуждала богиня Деметра по земле, проливая слезы.
Она всюду искала Персефону, всех просила о помощи, но никто не мог помочь ей в ее горе.
Наконец, уже на день она пришла к богу Гелиосу - Солнцу и стала со слезами молить его:- О, Гелиос !
Ты объезжаешь на златой колеснице 

In [41]:
final_2 = all_question(sents_2)

for key, value in final_2.items():
    print(key, ":", sents_2[key-1], "\n",value, "\n")


1 : Вдруг разверзлась земля, и на конях появился из земли в колеснице владыка царства теней умерших, Аид. 
 ['Вдруг разверзлась земля?', 'Кто такой Аид?', 'Чьим земля был?'] 

2 : Он схватил Персефону, поднял ее на свою колесницу и в мгновение ока скрылся на своих конях в недрах земли. 
 ['Он схватил Персефону?', 'Что делала Персефона?', 'Кто такая Персефона?', 'Что Онк схватил?'] 

3 : Только вскрикнуть успела Персефона. 
 ['Кто такая Персефона?', 'Что делала Персефона?', 'вскрикнуть успела Персефона?'] 

4 : Далеко разнесся крик ужаса дочери Деметры ; он донесся и до пучин, и до, Олимпа. 
 ['Чьим крик был?', 'Кто такой Олимп?', 'Кто был дочери Деметры?', 'Кто такая Деметра?', 'Далеко разнесся крик?', 'Что делал Олимп?'] 

5 : Никто не видел, как похитил Персефону Аид, видел лишь его бог Гелиос - Солнце. 
 ['Кто был бог Гелиос?', 'Кто такое Персефону Аид?', 'Кто такое Гелиос - Солнце?', 'Чьим бог был?', 'Никто не видел похитил?'] 

6 : Богиня Деметра услыхала крик Персефоны. 
 ['Кто б